# Groundwater Productivity Data Processing for Africa

This notebook processes groundwater productivity data for Africa by:

1. Loading and processing groundwater productivity data
2. Previewing processed data

In [1]:
# Import required libraries
import matplotlib.pyplot as plt
from shapely.geometry import Point
from tqdm import tqdm
import numpy as np
import geopandas as gpd
import pandas as pd
import sys
import os

# Add the project root to sys.path so we can import from Code.utils everywhere
project_root = os.path.abspath(os.path.join(os.getcwd(), '..', '..'))
if project_root not in sys.path:
    sys.path.insert(0, project_root)

from Code.utils.utility import load_config, resolve_path, ssa_iso, africa_iso_countries, africa_iso_countries_filtered

# Load configuration
config = load_config()

## 1. Load and process groundwater productivity data

In [2]:
# Read groundwater productivity data
input_path = resolve_path(config['Groundwater_Productivity_path'])
df_GW = pd.read_csv(input_path, sep=r'\s+')

# Create geometry for groundwater productivity points
geometry = [Point(xy) for xy in tqdm(zip(df_GW['X'], df_GW['Y']), desc="Creating geometries")]
gdf_GW = gpd.GeoDataFrame(df_GW, geometry=geometry)
gdf_GW.set_crs(epsg=4326, inplace=True)

# Map groundwater productivity categories to numeric values
value_map_cat = {'VH': 20.0, 'H': 12.5, 'M': 3.0, 'LM': 0.75, 'L': 0.3, 'VL': 0.1}
gdf_GW['GW_prod_cat_L_s'] = gdf_GW['GWPROD_V2'].map(value_map_cat)

# Save as GeoPackage
output_path = resolve_path(config['Groundwater_Prod_gpkg_path'])
gdf_GW.to_file(output_path)

# Reproject to Web Mercator (EPSG:3857)
gdf_GW = gdf_GW.to_crs(epsg=3857)

Creating geometries: 1019265it [00:06, 154293.85it/s]



## 2. Preview processed data

In [3]:
# Show a preview of the processed data
gdf_GW.head()

,X,Y,GWPROD_V2,geometry,GW_prod_cat_L_s
0,9.41405,37.251192,M,POINT (1047967.285 4474177.758),3.0
1,9.46405,37.251192,M,POINT (1053533.259 4474177.758),3.0
2,9.51405,37.251192,M,POINT (1059099.234 4474177.758),3.0
3,9.56405,37.251192,M,POINT (1064665.209 4474177.758),3.0
4,9.61405,37.251192,H,POINT (1070231.183 4474177.758),12.5
